In [21]:
import pandas as pd
import numpy as np

In [22]:
def modified_z_score(series):
    median = series.median()
    mad = np.median(np.abs(series - median))
    modified_z_scores = 0.6745 * (series - median) / mad
    return modified_z_scores

def remove_outliers_and_apply_thresholds(df, columns, thresholds, z_threshold=3.5):
    for col in columns:
        # Remove outliers using Modified Z-Score
        modified_z_scores = modified_z_score(df[col])
        df = df[np.abs(modified_z_scores) <= z_threshold]
        
        # Apply thresholds
        if col in thresholds:
            lower, upper = thresholds[col]
            df = df[(df[col] >= lower) & (df[col] <= upper)]
    
    return df

# Define thresholds for specific columns
thresholds = {
    "latitude": (-90, 90),
    "longitude": (-180, 180),
    "altitude": (-500, 12000),
    "course": (0, 360),
    "hacc": (0, 100),  # Assuming max horizontal accuracy of 100 meters
    "speed": (0, 300)  # Assuming max speed of 300 m/s
}



In [23]:
def merge_and_average(df):
    # Convert Timestamp to datetime and remove milliseconds
    df["Timestamp"] = pd.to_datetime(df["Timestamp"]).dt.floor("S")

    # Keep the relevant columns
    # Including extra columns: 'altitude', 'course', 'hacc', 'latitude', 'longitude', 'speed'
    df = df[["Timestamp", "X", "Y", "Z", "Type", "altitude", "course", "hacc", "latitude", "longitude", "speed"]].copy()

    # Create columns for each type
    types = df["Type"].unique()
    for t in types:
        for col in ["X", "Y", "Z"]:
            df.loc[:, f"{t}_{col}"] = df.apply(
                lambda row: row[col] if row["Type"] == t else None, axis=1
            )

    # Group by Timestamp and calculate mean for numeric columns
    df_grouped = df.groupby("Timestamp").mean(numeric_only=True).reset_index()

    # Explicitly list columns to retain
    columns_to_keep = ["Timestamp", "altitude", "course", "hacc", "latitude", "longitude", "speed"] + [
        f"{t}_{col}"
        for t in types
        for col in ["X", "Y", "Z"]
        if f"{t}_{col}" in df_grouped.columns
    ]
    df_grouped = df_grouped[columns_to_keep]

    # Round the numeric columns to the desired decimal places
    # Uncomment and modify the line below to set the desired decimal places
    df_grouped = df_grouped.round(6)

    return df_grouped

In [55]:
import os

files=os.listdir('./raw/')
for file in files:
    original_file = file
    data=pd.read_csv('./raw/'+original_file)
    '''
    注意，未来有标签，字符串类型
    '''
    columns_to_check = ["X", "Y", "Z", "altitude", "course", "hacc", "latitude", "longitude", "speed"]
    cleaned_data = remove_outliers_and_apply_thresholds(data, columns_to_check, thresholds)
    # Apply the function to the data
    cleaned_merged_data = merge_and_average(data)
    cleaned_merged_data = cleaned_merged_data.drop(['Position_X', 'Position_Y', 'Position_Z'], axis=1)
    cleaned_merged_data = cleaned_merged_data.bfill().interpolate()
    file_name=original_file.replace('-raw','')
    cleaned_merged_data.to_csv(file_name,index=False)


/home/taoyida/miniconda3/envs/ML4QS/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/taoyida/miniconda3/envs/ML4QS/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/taoyida/miniconda3/envs/ML4QS/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/taoyida/miniconda3/envs/ML4QS/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/taoyida/miniconda3/envs/ML4QS/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/taoyida/miniconda3/envs/ML4QS/lib/python3.8/site-packages/numpy/co